In [ ]:
import pandas as pd
import re
import time
from janitor import pivot_longer,pivot_wider
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats # for truncated normal distribution
import sklearn.model_selection # for train/test split and also model
import pickle # save model 
from sklearn import preprocessing # standardize inputs for neural network
import torch
from torch import nn
from torch.nn import functional as F

# Helper Functions
This section provides some helper functions for work with the truncated normal distribution and with pandas.

In [ ]:
# truncated normal function scaling a and b first
def trunc_normal(a_trunc,b_trunc,mu,sigma):
    a, b = (a_trunc - mu) / sigma, (b_trunc - mu) / sigma
    dist = stats.truncnorm(a=a,b=b,loc = mu,scale = sigma)
    return dist

# return negative log likelihood given certain parameters for tuncated normal distribution
def log_lik_trunc_normal(data,mu_test,sigma_int = .01,a_trunc = 0,b_trunc = 1):
    a_scaled = (a_trunc-mu_test)/sigma_int
    b_scaled = (b_trunc-mu_test)/sigma_int
    out = stats.truncnorm.nnlf(
        [
            a_scaled,
            b_scaled,
            mu_test,
            sigma_int
        ],
        data
    )
    return out

# distinct() function from R, assing to method for method chaining
def distinct_df(self,vars_to_distinct = 'Everything'):
    if vars_to_distinct == 'Everything':
        out = self.groupby(
            [i for i in self.columns],
            as_index = False
        ).size().drop('size',axis = 1)
    else:
        out = self.groupby(
            vars_to_distinct,
            as_index = False
        ).size().drop('size',axis = 1)
    return out

def distinct(self):
    return pd.unique(self)

pd.Series.distinct = pd.unique

pd.DataFrame.distinct = distinct_df

# Load Data and Add Features
There are some miscellaneous fields added here that were considered for use in this or related analyses.

In [ ]:
# get matches 2003-2023
years = range(2003,2024)
match_datasets = [
    pd.read_csv(f"https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_{i}.csv") for i in years
]
matches = pd.concat(match_datasets)

matches = matches.assign(
    match_id = matches['tourney_date'].astype('str') + '-' + matches['winner_id'].astype(str) + 
        '-' + matches['loser_id'].astype(str),
    year = list(map(lambda x: x[0:4],matches['tourney_date'].astype('str')))
).rename(
    columns = lambda x: x.replace('1st','first').replace('2nd','second').replace('l_','loser_'),
    inplace = False
).rename(
    columns = lambda x: x if x == 'draw_size' else x.replace('w_','winner_')
).assign(
    num_tiebreaks = lambda df: list(
        map(
            lambda x: len(re.findall('\\(',x)),
            df['score']
        )
    ),
    winner_tiebreaks_won = lambda df: list(
        map(
            lambda x: len(re.findall('7-6\\(',x)),
            df['score']
        )
    ),
    loser_tiebreaks_won = lambda df: list(
        map(
            lambda x: len(re.findall('6-7\\(',x)),
            df['score']
        )
    )
)


w_l_regex = re.compile('(winner|loser)')

# filter out retirements, no sv games
def not_retirement(score):
    out = [not 'RET' in i for i in score]
    return out

player_matches = matches.pivot_longer( # get player_matches
    column_names = [x for x in matches.columns if len(re.findall(w_l_regex,x)) != 0],
    names_to = ['result','.value'],
    names_pattern = r'(winner|loser)_(.+)'    
).rename( # convert to snake case
    columns = lambda x: re.sub(r'(\w)([A-Z])',r'\g<1>_\g<2>',x).lower()
).query( # no retirements, no walkovers, no missing sv_gms or svpt 
    '@not_retirement(score) and score != "W/O" and sv_gms != 0 and svpt != 0'
).query(# no carpet matches, no missing surface, no missing match stats, no missing age, no missing rank
    'surface != "Carpet" and not @pd.isna(surface) and not @pd.isna(ace)\
    and not @pd.isna(age) and not @pd.isna(rank)'
).fillna( # how to deal with NAs
    value = {
        'seed':'Unseeded', # treat unseeded as different category
        'entry':'NA', # treat no entry as separate, to create dummies without NA column
        'minutes':1e9, # placeholder, not going to include in model
        'hand':'U', # fill unknown for hand
        'ht': 'Unknown' # dont care about missing height
    }
).query( # remove odd hand entries and rounds that can't be ordered
    'hand not in ["A","U"] and round not in ["ER","RR"]'
)

In [ ]:
# add a bunch of features
intermediate_pm = player_matches.assign(
    rally_svpt = lambda df: df.svpt - df.ace - df.df,
    first_lost = lambda df: df.first_in - df.first_won,
    second_lost = lambda df: df.svpt - df.first_in - df.second_won,
    first_in_per = lambda df: df.first_in/df.svpt,
    first_won_per = lambda df: df.first_won/df.first_in,
    second_won_per = lambda df: df.second_won/(df.second_won+df.second_lost),
    hold_per = lambda df: (df.sv_gms - (df.bp_faced - df.bp_saved))/df.sv_gms,
    svptw = lambda df: df.first_won + df.second_won,
    svptw_per = lambda df: df.svptw/df.svpt,
    rally_svptw = lambda df: df.first_won + df.second_won - df.ace,
    rally_svptw_per = lambda df: df.rally_svptw/df.rally_svpt,
    pts_lost_per_svgm = lambda df: (df.first_lost + df.second_lost)/df.sv_gms,
    ace_per = lambda df: df.ace/df.svpt,
    df_per = lambda df: df['df']/df.svpt
).dropna() # a couple of issues where there are no second serve points


# function to change tourney_level
def change_tourney_level(tourney_level_og):
    if tourney_level_og == "M":
        return "Masters"
    elif tourney_level_og == "A":
        return "ATP 250/500"
    elif tourney_level_og == "G":
        return "Grand Slam"
    elif tourney_level_og == "F":
        return "Year End Final"
    elif tourney_level_og == "D":
        return "Davis Cup"
    
# function to bin rankings
def bin_rankings(x):
    if x <= 10:
        return "Top 10"
    elif x<= 25:
        return "Top 25"
    elif x <= 50:
        return "Top 50"
    elif x <= 100:
        return "Top 100"
    else:
        return "Outside Top 100"


# player_matches_wfeat
player_matches_wfeat = intermediate_pm.merge(
    intermediate_pm.rename(
        columns = lambda x: x if x == 'match_id' else 'opp_' + x,
        inplace = False
    ),
    how = 'left',
    on = 'match_id'
).query(
    'name != opp_name'
).assign(
    rtgms = lambda df: df.opp_sv_gms,
    aces_against = lambda df: df.opp_ace,
    break_per = lambda df: 1- df.opp_hold_per,
    bp_created = lambda df: df.opp_bp_faced,
    bp_conversion = lambda df: 1-(df.opp_bp_saved/df.opp_bp_faced),
    bp_per_gm = lambda df: df.bp_created/df.rtgms,
    rpts = lambda df: df.opp_svpt,
    rpw = lambda df: df.opp_svpt - df.opp_first_won - df.opp_second_won,
    rpw_per = lambda df: df.rpw/df.rpts,
    first_rpw_per = lambda df: 1-df.opp_first_won_per,
    second_rpw_per = lambda df: 1-df.opp_second_won_per,
    rally_rp = lambda df: df.opp_rally_svpt,
    rally_rpw = lambda df: df.rally_rp - df.opp_rally_svptw,
    rally_rpw_per = lambda df: df.rally_rpw/df.rally_rp,
    dominance_ratio = lambda df: df.rpw_per/(1-df.svptw_per),
    bp_face_freq = lambda df: df.bp_faced/df.svpt,
    bp_create_freq = lambda df: df.bp_created/df.rpts,
    ace_per_against = lambda df: df.opp_ace/df.opp_svpt
).filter(
    regex=r'^(?!opp).+',
    axis = 1
).merge(
    intermediate_pm.rename(
        columns = lambda x: x if x == 'match_id' else 'opp_' + x,
        inplace = False
    ).loc[
        :,
        ['match_id','opp_name','opp_rank','opp_rank_points']
    ],
    how = 'left',
    on = 'match_id'

).query(
    'name != opp_name'
).assign(
    rank_bin = lambda df: list(map(bin_rankings,df['rank'])),
    opp_rank_bin = lambda df: list(map(bin_rankings,df.opp_rank)),
    tourney_level = lambda df: list(map(change_tourney_level,df['tourney_level'])),
    # fix data issue with entry 
    entry = lambda df: list(map(lambda x: x if x != "Alt" else "ALT",df['entry'])),
    result = lambda df: list(
        map(
            lambda x: int(x == 'winner'),
            df['result']
        )
    ),
    year = lambda df: df['year'].astype(float)
)

In [ ]:
model_feats = [
    # id first and outcome
    'match_id', 'result',
    # fixed features about the match
    'year','tourney_level','surface','draw_size','best_of','round',
    'seed','entry','rank_points', 'rank_bin','opp_rank_points','opp_rank_bin','age',
    # features to be simulated - bounded percents trunc_normal
    'first_in_per','first_won_per','second_won_per',
    'rally_svptw_per','ace_per','df_per',
    'first_rpw_per','second_rpw_per','rally_rpw_per',
    # converted from counts to percentage so trunc normal
    'bp_face_freq','ace_per_against','bp_create_freq'    
]

trunc_normal_vars = [
    'first_in_per','first_won_per','second_won_per',
#     'hold_per',
    'rally_svptw_per','ace_per','df_per',
#     'break_per',
    'first_rpw_per',
    'second_rpw_per','rally_rpw_per','bp_face_freq','ace_per_against',
    'bp_create_freq'
]


# add dummy variables
model_df = pd.get_dummies(
    player_matches_wfeat.loc[
    :,
    model_feats
],
    columns = ['surface','tourney_level','draw_size','best_of',
              'round','seed','entry','rank_bin','opp_rank_bin'],
    dtype = float
)


# keep 2023 for testing
holdout_data = model_df.query(
    'year == 2023'
)
model_df = model_df.query('year != 2023')

# Stabilization of Career Statistics
This is for code considering the R^2 of match statistics over a player's career.

In [ ]:
# order matches based on round
def update_tourney_date(tourney_date,round_match):
    if round_match in ['BR','F']:
        out = str(tourney_date) + '7'
    elif round_match == 'SF':
        out = str(tourney_date) + '6'
    elif round_match == 'QF':
        out = str(tourney_date) + '5'
    elif round_match == 'R16':
        out = str(tourney_date) + '4'
    elif round_match == 'R32':
        out = str(tourney_date) + '3'
    elif round_match == 'R64':
        out = str(tourney_date) + '2'
    elif round_match == 'R128':
        out = str(tourney_date) + '1'
    return int(out)


pm_with_matchno = player_matches_wfeat.query('round not in ["RR","ER"]').assign(
    tourney_date = lambda df: list(
        map(
            lambda x,y: update_tourney_date(x,y),
            df['tourney_date'],
            df['round']
        )
    )
).groupby(
    'name',
    as_index = False
).apply( # get only players whose first match was in at least 2006 and played at least 50 matches
    lambda df: df if min(df.year) >= 2006 and df.shape[0]>=50 else None
).reset_index().drop(
    ['level_0','level_1'],
    axis = 1
).sort_values(['name','tourney_date']).loc[
    :,
    ['name','tourney_date','round',*trunc_normal_vars]
].groupby(
    'name',
    as_index = False
).apply(
    lambda df: df.assign(# get match number
        match_no = lambda x: x['tourney_date'].rank()
    )
).reset_index().drop(
    ['level_0','level_1'],
    axis = 1
)


stat_correlations = pm_with_matchno.pivot_longer(
    index = ['name','tourney_date','round','match_no'],
    names_to = 'stat_name',
    values_to = 'value'
).sort_values(['name','stat_name','match_no']).groupby(
    ['name','stat_name'],
    as_index = False
).apply(
    lambda df: df.assign( # get cumulative average of each statistic
        cum_avg_value = lambda x: x['value'].expanding(1).mean(),
        fin_value = lambda x: x['cum_avg_value'].tolist()[-1]
    )
).reset_index().drop(
    ['level_0','level_1'],
    axis = 1
).groupby(
    ['match_no','stat_name'],
    as_index = False
).apply( # make sure there is at least 1 match for a given match number
    lambda df: df if df.shape[0] > 1 else None
).groupby(
    ['match_no','stat_name'],
    as_index = False
).apply(
    lambda df: df.assign( # r^2 for cumulative average and final value
        corr = lambda x: np.corrcoef(x['cum_avg_value'],x['fin_value'])[0,1]**2,
        num_players = df.shape[0]
    ).distinct(['stat_name','match_no','corr','num_players'])
).reset_index().drop(
    ['level_0','level_1'],
    axis = 1
).query('num_players>=104').pivot_wider(
    index = ['match_no','num_players'],
    names_from = 'stat_name',
    values_from = 'corr'
)

In [ ]:
# single R-squared plot for first_won_per
ax = plt.plot(stat_correlations['first_won_per'],color = "#5C8FA3")
x_loc_of_vline = [stat_correlations['first_won_per']>=0.8][0].tolist().index(True)+1
plt.vlines(x = x_loc_of_vline,ymin = 0,ymax = 1,color = 'black',linestyles = 'dashed')
plt.ylim((0,1))
plt.title("First Serve Won % R-squared Cumulative vs Career")
plt.xlabel("Match Number")
plt.ylabel("R-squared")
plt.show()

In [ ]:
# R-squared plots for all inputs
fig,axs = plt.subplots(3,4,figsize = (20,10),sharex = False)
plt.setp(axs,ylim = (0,1))

fig.subplots_adjust(top = 0.99, bottom=0.01)
fig.subplots_adjust(hspace = 0.4)

colors = ["#79BCD6", "#798DD6", "#9379D6", "#C279D6", 
          "#D679BC", "#D6798D", "#D69379", "#D6C279", 
          "#BCD679", "#8DD679", "#79D693", "#79D6C2"]

# plot all lines
count = 0
for i in range(3):
    for j in range(4):
        axs[i,j].plot(stat_correlations[trunc_normal_vars[count]],color = colors[count])
        x_loc_of_vline = [stat_correlations[trunc_normal_vars[count]]>=0.8][0].tolist().index(True)+1
        axs[i,j].vlines(x = x_loc_of_vline,ymin = 0,ymax = 1,color = 'black',linestyles = 'dashed')
        axs[i,j].set_title(trunc_normal_vars[count])
        count+=1
plt.show()

# Distribution Plots

In [ ]:
# distribution for first_won_per
ax = player_matches_wfeat['first_won_per'].hist(bins = 30, color = "#5C8FA3")
ax.xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))
ax.grid(False)
plt.title("Distribution of First Serve Won %")
plt.xlim((0,1))
plt.show()

In [ ]:
# distributions for all inputs
fig,axs = plt.subplots(3,4,figsize = (20,10),sharex = False)
plt.setp(axs,xlim = (0,1))


fig.subplots_adjust(top = 0.99, bottom=0.01)
fig.subplots_adjust(hspace = 0.4)

colors = ["#79BCD6", "#798DD6", "#9379D6", "#C279D6", 
          "#D679BC", "#D6798D", "#D69379", "#D6C279", 
          "#BCD679", "#8DD679", "#79D693", "#79D6C2"]

# plot all distributions
count = 0
for i in range(3):
    for j in range(4):
        axs[i,j].hist(player_matches_wfeat[trunc_normal_vars[count]],bins = 30,color = colors[count])
        axs[i,j].set_title(trunc_normal_vars[count])
        axs[i,j].xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))
        count+=1
plt.show()

In [ ]:
# distributions for hold and break percentage
fig,axs = plt.subplots(1,2,figsize = (20,10))
plt.setp(axs,xlim = (0,1))

axs[0].hist(player_matches_wfeat['hold_per'],bins = 30,color = "#79BCD6")
axs[0].set_title('hold_per')
axs[0].xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))

axs[1].hist(player_matches_wfeat['break_per'],bins = 30,color = "#D69379")
axs[1].set_title('break_per')
axs[1].xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))
plt.show()

# Modeling
This section includes code used to build the post-match win probability models with subsections for each variant.

In [ ]:
X_train, X_test,y_train,y_test = sklearn.model_selection.train_test_split(
    model_df.drop(['result'],axis = 1),
    model_df['result'],
    test_size=0.2,
    random_state = 33 # for reproducibility
)

random_seed = 54
np.random.seed(random_seed)

## Multilayer Perceptron

In [ ]:
# scale values for nn, only use training data
scaler = preprocessing.StandardScaler().fit(
    X_train.loc[
        :,
        [*trunc_normal_vars,'year','age',
         'rank_points','opp_rank_points']
    ]
)

# create standardized copies of data for neural networks
X_train_nn = X_train.copy()
X_test_nn = X_test.copy()

X_train_nn.loc[:,scaler.feature_names_in_] = scaler.transform(
    X_train_nn.loc[:,scaler.feature_names_in_]
)

X_test_nn.loc[:,scaler.feature_names_in_] = scaler.transform(
    X_test_nn.loc[:,scaler.feature_names_in_]
)

# create dataloaders for train and validation datasets
np.random.seed(random_seed)
X_train_nn_tens = torch.tensor(X_train_nn.iloc[:,1:].values).float()
y_train_nn = torch.tensor(pd.get_dummies(y_train).values).float() # make this two-dimensional for cross-entropy loss
X_val_nn_tens = torch.tensor(X_test_nn.iloc[:,1:].values).float()
y_val_nn = torch.tensor(pd.get_dummies(y_test).values).float()

nn_train = torch.utils.data.TensorDataset(X_train_nn_tens,y_train_nn)
train_loader = torch.utils.data.DataLoader(nn_train,batch_size = 32,shuffle=True)
nn_val = torch.utils.data.TensorDataset(X_val_nn_tens,y_val_nn)
val_loader = torch.utils.data.DataLoader(nn_val,batch_size = 32,shuffle=True)

In [ ]:
# model class
class MyMLP(nn.Module):
    def __init__(self,num_hidden_layers,num_neurons,num_outputs = 2,lr = 0.05):
        # call constructor of parent class
        super().__init__()
        self.lr = lr
        self.net = nn.Sequential()
        
        # add hidden layers
        for i in range(num_hidden_layers):
            self.net.add_module('layer'+str(i),nn.LazyLinear(num_neurons[i]))
            self.net.add_module('relu'+str(i),nn.ReLU())
            self.net.add_module('dropout'+str(i),nn.Dropout(.25))
           
        # add output layer
        self.net.add_module('output',nn.LazyLinear(num_outputs))   
        
    # define forward method to calculate output
    def forward(self,x):
        return self.net(x)
    
    # cross-entropy loss for two classes
    def loss(self,y_hat,y):
        return F.cross_entropy(y_hat,y)
    
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), self.lr)
    
    # predict method to output probabilities
    def predict(self,x):
        return nn.Softmax(dim=1)(self.net(x))
        
    # method to get accuracy on a batch
    def get_accuracy(self,batch):
        preds = self.predict(batch[0]).argmax(axis = 1)
        actual = batch[1].argmax(axis = 1)
        return sum(preds == actual)/len(preds == actual)
        
    # calculate loss for training step
    def training_step(self,batch):
        out_loss = self.loss(self(batch[0]),batch[1])
        return out_loss
    
    # calculate loss for validation step
    def val_step(self,batch):
        out_loss = self.loss(self(batch[0]),batch[1])
        return out_loss
    
class MyTrainer:
    
    def __init__(self,num_epochs,early_stopping = None):
        self.num_epochs = num_epochs
        self.early_stopping = early_stopping

    def fit(self,model,train_dataloader,val_dataloader,plot_loss = False):
        # assign things
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        
        # get optimizer
        self.optim = model.configure_optimizers()
        self.epoch = 0
        
        
        # attributes for plotting
        self.epoch_list = []
        self.train_loss_list = []
        self.val_loss_list = []
        self.val_acc_list = []

        if plot_loss: # not implemented/working
            # first stuff for plotting
            plt.ion()
            fig,ax = plt.subplots(1,1)
            fig.show()
            fig.canvas.draw()
        
        for self.epoch in range(self.num_epochs): # iterate through epoch 
            
            self.best_loss = np.inf
            
            # early stopping if val_loss hasn't improved after x rounds
            if isinstance(self.early_stopping,int) and self.epoch >= self.early_stopping:
                # starting point needs to be early_stopping+1 back so to compare the following early_stopping points
                start_point = self.epoch-self.early_stopping-1
                end_point = self.epoch+1
                should_stop = [self.val_loss_list[start_point] < i for i in self.val_loss_list[(start_point+1):end_point]]
                if sum(should_stop)==len(should_stop):
                    print(f'Early stopping, validation loss has not improved after {self.early_stopping} rounds')
                    break
                
            self.fit_epoch()
            self.epoch_list.append(self.epoch+1)
            
            if plot_loss: # not implemented/working
                # plot updates
                ax.clear()
                ax.plot(self.epoch_list,self.train_loss_list,label = 'train_loss',color = 'blue')
                ax.plot(self.epoch_list,self.val_loss_list,label = 'val_loss',color = 'orange')
                ax.plot(self.epoch_list,self.val_acc_list,label = 'val_acc',color = 'green')
                ax.legend(loc = 'best')
                fig.canvas.draw()
                plt.pause(.1)
        
        # plot progression at end
        fig,ax = plt.subplots(1,1)
        ax.plot(self.epoch_list,self.train_loss_list,label = 'train_loss',color = 'blue')
        ax.plot(self.epoch_list,self.val_loss_list,label = 'val_loss',color = 'orange')
        ax.plot(self.epoch_list,self.val_acc_list,label = 'val_acc',color = 'green')
        ax.legend(loc = 'best')
        ax.set_title('Train and Val Loss')
            
    # feed forward and backpropogation process
    def fit_epoch(self):
        cum_train_loss = []
        self.model.train()
        for batch in self.train_dataloader:
            loss = self.model.training_step(batch)
            self.optim.zero_grad()
            with torch.no_grad():
                loss.backward()
                self.optim.step()
            cum_train_loss.append(loss.detach())
        if self.val_dataloader is None:
            return
        cum_val_loss = []
        cum_val_acc = []
        self.model.eval()
        for batch in self.val_dataloader:
            with torch.no_grad():
                self.model.val_step(batch)
            cum_val_loss.append(self.model.val_step(batch).detach())
            cum_val_acc.append(self.model.get_accuracy(batch))
        
        # for early stopping
        if self.model.val_step(batch).detach() < self.best_loss:
            torch.save(self.model.state_dict(),'Final Models/MLP Best Val Tune (2024.05.28 NS).pkl')
            # for debugging
            self.best_tune_params = self.model.parameters()
        
        # store all loss and acc for plotting
        self.train_loss_list.append(np.mean(cum_train_loss))
        self.val_loss_list.append(np.mean(cum_val_loss))
        self.val_acc_list.append(np.mean(cum_val_acc))
        
        print(f'epoch {self.epoch+1} - train_loss: {np.mean(cum_train_loss):.4f}, val_loss: {np.mean(cum_val_loss):.4f}, val_acc: {np.mean(cum_val_acc):.4f}')

In [ ]:
# train a model with one hidden layer
np.random.seed(random_seed)
test_mlp = MyMLP(num_hidden_layers = 1,num_neurons = [33],lr = 0.01)
trainer = MyTrainer(num_epochs=250,early_stopping=20)
trainer.fit(test_mlp, train_dataloader=train_loader,val_dataloader=val_loader)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
np.random.seed(random_seed)
# initial random forest
rf_init = RandomForestClassifier(
    n_estimators = 500,
    criterion = 'gini',
    max_depth = 25, # default none
    verbose = 0
)
rf_init.fit(
    X_train.iloc[:,1:],y_train
)


# grid search cross-validation
rf_param_grid = {
    'n_estimators':[100,200,300,400,500],
    'max_depth':[5,10,15,20,25]
}

rf_gs = sklearn.model_selection.GridSearchCV(rf_init,rf_param_grid,cv = 5)
rf_gs.fit(
    X_train.iloc[:,1:],
    y_train
)

## XGBoost

In [ ]:
import xgboost as xgb
np.random.seed(random_seed)

# build initial model
xgb_model_sklearn_init = xgb.XGBRegressor(
    max_depth = 5,
    n_estimators = 400,
    eta = 0.1,
    objective = 'binary:logistic',
    eval_metric = 'logloss'
)
xgb_model_sklearn_init.fit(
    X_train.iloc[:,1:],
    y_train
)

# get accuracy of initial model
preds = xgb_model_sklearn_init.predict(X_test.iloc[:,1:])
sklearn.metrics.accuracy_score(
    y_pred= [round(i) for i in preds],
    y_true = y_test
)

# tuning the hyperparameters with grid search and cross-validation
param_grid = {
    'max_depth':list(range(5,9)),
    'n_estimators':[400,500,600],
    'eta':[0.1,0.2,0.3],
    'objective':['binary:logistic'],
    'eval_metric':['logloss']
}
xgb_gs = sklearn.model_selection.GridSearchCV(xgb_model_sklearn_init,param_grid,cv = 5)
xgb_gs.fit(
    X_train.iloc[:,1:],
    y_train
)

# Maximum Likelihood Estimation
During the project, it was easier to save the parameters of the distribution in a csv and load them in to parse into distributions as a method of checkpointing progress.

In [ ]:
# function to get parameters of maximum likelihood distribution
def get_trunc_normal_dist(list_of_vals,is_dominance_ratio = "No"):
    mu_int = np.arange(
        np.median(list_of_vals)-.1,
        np.median(list_of_vals)+.1,
        .01
    )
    sigma_base = np.arange(.01,0.3,.01)
    int_param_grid = [[i,j] for i in mu_int for j in sigma_base]
    neg_log_lik_list = [
        log_lik_trunc_normal(
            data=list_of_vals,
            mu_test=i[0],
            sigma_int = i[1]
        ) for i in int_param_grid
    ]
    
    win_index_int = (neg_log_lik_list == min(neg_log_lik_list)).tolist().index(True)
    
    return int_param_grid[win_index_int]

In [ ]:
# player distributions for truncated normal stats
player_distributions = player_matches_wfeat.merge(
    holdout_data.loc[:,['match_id','result']].assign(flag = 1),
    how = 'left'
).query(
    '@pd.isna(flag)'
).drop('flag',axis = 1).groupby(
    ['name','surface'],
    as_index = False
).aggregate(
    {col:((lambda x: list(x)) if col in trunc_normal_vars else len) for col in [*trunc_normal_vars,'tourney_name']}
).rename(
    columns = {'tourney_name':'num_matches'}
).query(
    'num_matches >= 75 or (num_matches >= 60 and surface == "Grass")'
).pivot_longer(
    index = ['name','surface','num_matches'],
    names_to = 'stat_name',
    values_to = 'value'
).assign(
    dist_params = lambda df: list(
        map(
            get_trunc_normal_dist,
            df['value']
        )
    )
).pivot(
    columns = 'stat_name',
    index = ['name','num_matches','surface'],
    values = 'dist_params'
).reset_index()
player_distributions.to_csv("Player Distributions (2024.04.15 NS).csv",index = False)

# reload in player distributions and parse into distributions
player_distributions = pd.read_csv("Player Distributions (2024.04.15 NS).csv").apply(
    lambda df: df if df.name not in trunc_normal_vars else list(
        map(
            lambda x: x.strip('][').split(','),
            df
        )
    )
).apply(
    lambda df: df if df.name not in trunc_normal_vars else list(
        map(
            lambda x: trunc_normal(0,1,float(x[0]),float(x[1])),
            df
        )
    )
)

In [ ]:
# distributions for each rank bin against other rank bins
rank_bin_distributions = player_matches_wfeat.merge(
    holdout_data.loc[:,['match_id','result']].assign(flag = 1),
    how = 'left'
).query(
    '@pd.isna(flag)'
).drop('flag',axis = 1).groupby(
    ['rank_bin','opp_rank_bin','surface'],
    as_index = False
).aggregate(
    {col:((lambda x: list(x)) if col in trunc_normal_vars else len) for col in [*trunc_normal_vars,'tourney_name']}
).rename(
    columns = {'tourney_name':'num_matches'}
).pivot_longer(
    index = ['rank_bin','opp_rank_bin','surface','num_matches'],
    names_to = 'stat_name',
    values_to = 'value'
).assign(
    dist_params = lambda df: list(
        map(
            get_trunc_normal_dist,
            df['value']
        )
    )
).pivot(
    columns = 'stat_name',
    index = ['rank_bin','opp_rank_bin','num_matches','surface'],
    values = 'dist_params'
).reset_index()
rank_bin_distributions.to_csv("Rank Bin Distributions (2024.04.15 NS).csv",index = False)


rank_bin_distributions = pd.read_csv("Rank Bin Distributions (2024.04.15 NS).csv").apply(
    lambda df: df if df.name not in trunc_normal_vars else list(
        map(
            lambda x: x.strip('][').split(','),
            df
        )
    )
).apply(
    lambda df: df if df.name not in trunc_normal_vars else list(
        map(
            lambda x: trunc_normal(0,1,float(x[0]),float(x[1])),
            df
        )
    )
)

# Evaluating the Method

In [ ]:
# baseline naive predictions based on rank
naive_preds = holdout_data.assign(
    pred_result = lambda df: list(
        map(
            lambda x,y:1 if x>y else 0,
            df['rank_points'],
            df['opp_rank_points']
        )
    )
)

naive_rank_acc = sklearn.metrics.accuracy_score(
    y_pred= round(naive_preds['pred_result']),
    y_true = naive_preds['result']
)

In [ ]:
# function to return accuracy from num_sims, differs a bit for neural network
def eval_num_sims(num_sims,model_param,is_nn = False):
    np.random.seed(random_seed)
    # join distributions then run sims
    holdout_sim_data_int = pd.concat(
        [
            holdout_data.merge(# player distributions
                player_matches_wfeat.loc[:,['match_id','result','surface','name']],
                how = 'left'
            ).drop(trunc_normal_vars,axis = 1).merge(
                player_distributions.drop(['hold_per','break_per'],axis = 1),
                how = 'left'
            ).query(
                "not @pd.isna(first_in_per)"
            ),
            holdout_data.merge(# rank distributions, only join if no player distribution match
                player_matches_wfeat.loc[:,['match_id','result','surface','name','rank_bin','opp_rank_bin']],
                how = 'left'
            ).drop(trunc_normal_vars,axis = 1).merge(
                player_distributions.drop(['hold_per','break_per'],axis = 1),
                how = 'left'
            ).query(
                "@pd.isna(first_in_per)"
            ).drop(trunc_normal_vars,axis = 1).merge(
                rank_bin_distributions.drop(['hold_per','break_per'],axis = 1),
                how = 'left',
                on = ['rank_bin','opp_rank_bin','surface']
            )
        ]
    ).apply(
        lambda df: df if df.name not in trunc_normal_vars else list(
            map( # get simulations using rvs method
                lambda x: x.rvs(num_sims),
                df
            )
        )
    ).explode(trunc_normal_vars).apply(
        lambda df: df if df.name not in trunc_normal_vars else df.astype(float)
    )
    
    # standardize columns if using neural network
    if is_nn:
        holdout_sim_data_int.loc[:,scaler.feature_names_in_] = scaler.transform(
            holdout_sim_data_int.loc[:,scaler.feature_names_in_]
        )
        preds_int = model_param.predict(
            torch.tensor(
                holdout_sim_data_int.loc[
                    :,
                    X_train.columns[1:]
                ].values
            ).float()
        ).argmax(axis = 1).detach().numpy()
    else:
         # get predictions for simulations
        preds_int = model_param.predict(
            holdout_sim_data_int.loc[
                :,
                X_train.columns[1:]
            ]
        )
    
    holdout_sim_data_int['pred_result'] = preds_int
    
    # for each match, get average predicted results over the simulations
    actual_preds_int = holdout_sim_data_int.groupby(
        ['match_id','result'],
        as_index = False
    ).aggregate(
        pred_result = ('pred_result',lambda x: sum(round(x))/len(x))
    )
    
    # normalize the predictions for the match
    further_analysis_int = player_matches_wfeat.query(
        'year == 2023'
    ).loc[
        :,
        ['match_id','name','tourney_name','round','result']
    ].merge(
        actual_preds_int.loc[:,['match_id','pred_result','result']]
    ).sort_values(
        'match_id'
    ).merge(
        actual_preds_int.groupby('match_id',as_index = False).aggregate(tot_result = ('pred_result',sum))
    ).assign(
        norm_pred_result = lambda df: df['pred_result']/df['tot_result']
    )
    
    acc = sklearn.metrics.accuracy_score(
        y_pred= round(further_analysis_int['norm_pred_result']),
        y_true = further_analysis_int['result']
    )
    
    print(f'done with {num_sims} sims')
    
    return acc


In [ ]:
# evaluate accuracy for number of sims across the three models
sim_testing_xgb = pd.DataFrame(
    {
        'num_sims':[100,300,500,750,1000,2000,3000]
    }
).assign(
    accuracy_sims = lambda df: list(
        map(
            lambda x: eval_num_sims(x,xgb_gs),
            df.num_sims
        )
    )
)
print('xgb done')
sim_testing_rf = pd.DataFrame(
    {
        'num_sims':[100,300,500,750,1000,2000,3000]
    }
).assign(
    accuracy_sims = lambda df: list(
        map(
            lambda x: eval_num_sims(x,rf_gs),
            df.num_sims
        )
    )
)
# print('rf done')
sim_testing_nn = pd.DataFrame(
    {
        'num_sims':[100,300,500,750,1000,2000,3000]
    }
).assign(
    accuracy_sims = lambda df: list(
        map(
            lambda x: eval_num_sims(x,final_mlp,is_nn=True),
            df.num_sims
        )
    )
)
print('nn done')

In [ ]:
# plot performance of three models over different number of sims
fig,ax = plt.subplots()
x_loc = np.arange(7)
col_width = 0.15

# plot three models
ax.bar(x_loc,sim_testing_nn['accuracy_sims'],color = '#5C8FA3',width = col_width,label = 'MLP')
ax.bar(x_loc+col_width,sim_testing_rf['accuracy_sims'],color = '#A35C8F',width = col_width,label = 'RF')
ax.bar(x_loc+2*col_width,sim_testing_xgb['accuracy_sims'],color = '#8FA35C',width = col_width,label = 'XGBoost')

plt.xticks(ticks = x_loc + col_width,labels=sim_testing_rf['num_sims'])
plt.ylim((0.60,0.66))
plt.title('Accuracy of Model Predictions vs Naive Ranking Approach')
plt.xlabel('Number of Simulations')
plt.ylabel('Accuracy')
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))
plt.legend(loc='best')

# add naive stuff
plt.hlines(y =naive_rank_acc,xmin = -0.5,xmax = 7.35,color = '#C79038')
plt.text(6.5,naive_rank_acc+.0005,str(round(naive_rank_acc*100,2))+'%')
plt.show()

## Extra analysis by surface and distributions

In [ ]:
# draw 100 random samples for analysis by surface and number of players with distribution
np.random.seed(random_seed)
holdout_sim_data = pd.concat(
    [
        holdout_data.merge(# player distributions
            player_matches_wfeat.loc[:,['match_id','result','surface','name']],
            how = 'left'
        ).drop(trunc_normal_vars,axis = 1).merge(
            player_distributions.drop(['hold_per','break_per'],axis = 1),
            how = 'left'
        ).query(
            "not @pd.isna(first_in_per)"
        ),
        holdout_data.merge(# rank distributions
            player_matches_wfeat.loc[:,['match_id','result','surface','name','rank_bin','opp_rank_bin']],
            how = 'left'
        ).drop(trunc_normal_vars,axis = 1).merge(
            player_distributions.drop(['hold_per','break_per'],axis = 1),
            how = 'left'
        ).query(
            "@pd.isna(first_in_per)"
        ).drop(trunc_normal_vars,axis = 1).merge(
            rank_bin_distributions.drop(['hold_per','break_per'],axis = 1),
            how = 'left',
            on = ['rank_bin','opp_rank_bin','surface']
        )
    ]
).apply(
    lambda df: df if df.name not in trunc_normal_vars else list(
        map( # get simulations using rvs method
            lambda x: x.rvs(100),
            df
        )
    )
).explode(trunc_normal_vars).apply(
    lambda df: df if df.name not in trunc_normal_vars else df.astype(float)
)

# get predicted results using random forest model
preds_hold = rf_gs.predict(
    holdout_sim_data.loc[
        :,
        X_train.columns[1:]
    ]
)
    
preds_hold = np.array(
    [round(i) for i in preds_hold]
)

holdout_sim_data['pred_result'] = preds_hold
act_pred_data = holdout_sim_data.groupby(
    ['match_id','result'],
    as_index = False
).aggregate(
    pred_result = ('pred_result',lambda x: sum(round(x))/len(x))
)


# normalize the probabilities from random forest results
norm_wp = player_matches_wfeat.query(
    'year == 2023'
).loc[
    :,
    ['match_id','name','tourney_name','surface','round','result']
].merge(
    act_pred_data.loc[:,['match_id','pred_result','result']]
).sort_values(
    'match_id'
).merge(
    act_pred_data.groupby('match_id',as_index = False).aggregate(tot_result = ('pred_result',sum))
).assign(
    norm_pred_result = lambda df: df['pred_result']/df['tot_result']
)


In [ ]:
# random forest model performance by surface
sim_surface_acc = norm_wp.assign(
    rounded_norm_prob = lambda df: round(df.norm_pred_result)
).groupby(
    'match_id',
    as_index = False
).apply(
    lambda df: df.assign(
        acc = lambda df2: df2.rounded_norm_prob == df2.result 
    )
).distinct(
    ['match_id','acc','surface']
).groupby(
    'surface',
    as_index = False
).aggregate(
    sim_acc = ('acc',np.mean)
)

# naive predictions by surface
naive_surface_acc = naive_preds.merge(
    player_matches_wfeat.distinct(['match_id','surface']),
    how='left',
    on='match_id'
).distinct(
    ['match_id','surface','result','pred_result']
).assign(
    acc = lambda df: df.result == df.pred_result
).distinct(['match_id','surface','acc']).groupby(
    'surface',
    as_index = False
).aggregate(
    naive_acc = ('acc',np.mean)
)

In [ ]:
# plot simulation approach vs rank-based across surfaces
fig,ax = plt.subplots()
ax.bar([0,1,2],sim_surface_acc['sim_acc'],width = 0.3,label = '100 Simulations',color = '#5C8FA3')
ax.bar([.3,1.3,2.3],naive_surface_acc['naive_acc'],width = 0.3, label = 'Rank-Based Approach',color = '#A3705C')
plt.xticks(ticks = [0.15,1.15,2.15],labels=sim_surface_acc['surface'])
plt.ylim((0.60,0.66))
plt.legend(loc='best')
plt.ylabel('Accuracy')
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))
plt.title('Simulation Accuracy Across Surfaces')
plt.show()

In [ ]:
# plot simulation performance based on whether the players in the match had their own distributions to draw from
perf_by_dist_pres = norm_wp.assign(
    rounded_norm_prob = lambda df: round(df.norm_pred_result)
).merge(
    player_distributions.loc[:,['name','surface']].assign(player_flag = 1),
    how = 'left',
    on = ['name','surface']
).fillna(0).assign(
    acc = lambda df: df.rounded_norm_prob == df.result
).groupby(
    ['match_id','acc'],
    as_index = False
).aggregate(
    num_player_dist = ('player_flag',sum)
).groupby(
    'num_player_dist',
    as_index = False
).aggregate(
    sim_acc = ('acc',np.mean)
)

# plot
fig,ax = plt.subplots()
ax.bar([str(int(i)) for i in perf_by_dist_pres['num_player_dist']],perf_by_dist_pres['sim_acc'],color = '#5C8FA3')
plt.ylim((0.60,0.68))
plt.xlabel('Number of Players with Distribution')
plt.ylabel('Accuracy')
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax = 1))
plt.title('Simulation Accuracy by Number of Players with Distribution')
plt.show()